<a href="https://colab.research.google.com/github/zamalex/BakingFinal/blob/master/pamap2_transformer_hier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn.metrics
from tensorflow.keras.utils import plot_model
import datetime


In [2]:
%load_ext tensorboard


In [3]:
import pandas as pd
import numpy as np
from sklearn import metrics
import h5py
from sklearn.model_selection import train_test_split

def read_h5(path):
        f = h5py.File(path, 'r')
        X = f.get('inputs')
        y = f.get('labels')
        #print(type(X))
        #print(type(y))
        X = np.array(X)
        y = np.array(y)
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)
        return x_train, x_test, y_train, y_test
        #print("X = ", X.shape)
        #print("y =",y.shape)
        #print(self.x_train.shape)
        #print(self.x_test.shape)
        #print(self.x_train.shape)
        #return X,y
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "drive/MyDrive/pamap_scaled3.h5"

x_train, x_test, y_train, y_test=read_h5(path)
x_train.shape, y_train.shape,x_test.shape, y_test.shape

((10086, 25, 40), (10086,), (4323, 25, 40), (4323,))

# Some methods and data which will be used below

In [ ]:
# (i).   Mapping labels to their resp. classes
# (ii).  Mapping classes to their resp. labels

label_to_class = {
    1  : 'WALKING',
    2  : 'WALKING_UPSTAIRS',
    3  : 'WALKING_DOWNSTAIRS',
    4  : 'SITTING',
    5  : 'STANDING',
    6  : 'LAYING',
    7  : 'STAND_TO_SIT',
    8  : 'SIT_TO_STAND',
    9  : 'SIT_TO_LIE',
    10 : 'LIE_TO_SIT',
    11 : 'STAND_TO_LIE',
    12 : 'LIE_TO_STAND',
    np.nan : np.nan
}
class_to_label = {
    'WALKING' : 1,
    'WALKING_UPSTAIRS' : 2,
    'WALKING_DOWNSTAIRS' : 3,
    'SITTING' : 4,
    'STANDING' : 5,
    'LAYING' : 6,
    'STAND_TO_SIT' : 7,
    'SIT_TO_STAND' : 8,
    'SIT_TO_LIE' : 9,
    'LIE_TO_SIT' : 10,
    'STAND_TO_LIE' : 11,
    'LIE_TO_STAND' : 12,
    np.nan : np.nan
}

In [5]:
# (i).   Removing data-points where y and x values is null

# Other methods can be
# ffill (forward fill) => fills using forward points
# bfill (backward fill) => using backward points
# interpolate

def remove_null(xdata,ydata):
    xdata = xdata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]
    xdata = xdata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]

    return xdata,ydata

In [6]:
# normalize xdata using sklearn.preprocessing.StandardScaler and returns
# scaler object to use it furthur for testing data

# Each axis of each sensor has different min, max, I scaled according to them seperately
# Initial shape == (None,128,2,3)
# changed to (None , 6) :-
# reshape to (None,128,6) -> swapaxis(0,2) -> reshape(6,-1) -> transpose
# Fit scaler OR transform according to scaler

# Reverse above process to get back oiginal data
# transpose -> reshape(6,128,None) -> swapaxes(0,2) -> reshape(None,128,2,3)

def get_scaler(xdata):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    scaler = MinMaxScaler(feature_range = (-1,1))
    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    scaler.fit(xdata)
    return scaler

def scale_data(xdata,scaler):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    xdata = scaler.transform(xdata)
    xdata = xdata.T.reshape(sensor*axis,timestamp,row)
    xdata = np.swapaxes(xdata,0,2).reshape(row,timestamp,sensor,axis)

    return xdata

In [7]:
# takes in location, exp no., user no., start and end(end point is excluded from reading i.e lastpoint+1) point
# ,overlap array, and returns xdata and ydata

def create_windows(location, exp, user, start, end, activity, length, overlap):

    acc_file  = location + '/acc_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'
    gyro_file = location + '/gyro_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'

    acc_data  = np.loadtxt(acc_file)
    gyro_data = np.loadtxt(gyro_file)

    xtrain = []
    ytrain = []

    while (start + length <= end) :

        stop = start + length
        window = []

        while start != stop :
            window.append( [acc_data[start] , gyro_data[start]] )
            start += 1

        xtrain.append(window)
        ytrain.append(activity)

        start = stop - overlap[activity-1]

    return xtrain,ytrain

In [8]:
# location == location of file
# lenght == lenght of window
# overlap == array of overlaps of size == number of unique activities
# overlap depends on activity so as to extract more data from a particular class if needed


# (i).   Loading labels.txt as labels
# (ii).  Iterating in labels and calling create_windows on acceleration file, extending returned data in xtrain, ytrain
# (iii). Iterating in labels and calling create_windows on gyroscope file, extending returned data in xtrain, ytrain

def prepare_data(location,length = 128,overlap = [64]*12):

    xdata = []
    ydata = []

    labels = np.loadtxt(location+'/labels.txt',dtype = 'uint32')

    for exp,user,activity,start,end in labels :

        xtemp , ytemp = create_windows(location, exp, user, start, end+1, activity, length, overlap)
        xdata.extend(xtemp)
        ydata.extend(ytemp)

    return np.array(xdata),np.array(ydata)

In [9]:
# (i). Finds max element index sets its 1 and sets remaining 0
#      for each row

def to_categorical(ydata):

    for i in range(len(ydata)):
        j = ydata[i].argmax()
        for k in range(len(ydata[i])):
            ydata[i][k] = (k == j)
    return ydata

In [55]:
              #1: 'lying',# no change in index
              #2: 'sitting',# no change in index
              #3: 'standing',# no change in index


              #4: 'walking',# no change in index
              #5: 'running',# no change in index
              #8: 'ascending_stairs', # new index:8
              #9: 'descending_stairs', # new index:9

              #0: 'rope_jumping' # new index: 0
              #6: 'cycling',# no change in index
              #7: 'Nordic_walking',# no change in index
              #10: 'vacuum_cleaning', # new index:10
              #11: 'ironing'

def one_hot_encoded(ydata):
    ydata = OneHotEncoder().fit_transform(ydata.reshape(len(ydata),1))
    ydata = np.asarray(ydata.todense())
    return ydata

def splitData(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==4)|(ytrain==5)|(ytrain==8)|(ytrain==9)]
    ytrain = ytrain[(ytrain==4)|(ytrain==5)|(ytrain==8)|(ytrain==9)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==0) | (ytrain==6)|(ytrain==7)|(ytrain==10)|(ytrain==11)]
    ytrain = ytrain[(ytrain==0) | (ytrain==6)|(ytrain==7)|(ytrain==10)|(ytrain==11)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)
  testlabels = ytest
  #scaler = get_scaler(xtrain)
  #xtrain = scale_data(xtrain,scaler)
  #xtest  = scale_data(xtest,scaler)

  ytrain = one_hot_encoded(ytrain)
  ytest = one_hot_encoded(ytest)
  return xtrain,xtest,ytrain,ytest,testlabels

def splitData2(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)

  return xtrain,xtest,ytrain,ytest


def resampleData(x,y):
  x = x.reshape(5773,768)
  df = pd.DataFrame(x)

  from imblearn.over_sampling import RandomOverSampler
  from collections import Counter
  from sklearn.datasets import make_classification
  from imblearn.over_sampling import SMOTE

  oversample = SMOTE()
  df, y = oversample.fit_resample(df, y)
  df=df.to_numpy()
  x=df.reshape(12228,128,6)
  x=df.reshape(12228,128,2,3)
  return x,y

# Data Preparation

## Loading data from files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import tensorflow as tf
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/

## Handling missing data and Normalizing data

In [14]:
xtrain = x_train
ytrain=y_train
xtest= x_test
ytest= y_test

xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((10086, 25, 40), (10086,), (4323, 25, 40), (4323,))

In [20]:
X = np.concatenate((xtrain, xtest), axis=0)
Y = np.concatenate((ytrain, ytest), axis=0)
X.shape, Y.shape

((14409, 25, 40), (14409,))

**Hier split and training**

**Dynamic**

In [56]:
dynamicxtrain,dynamicxtest,dynamicytrain,dynamicytest,dynamiclabels = splitData(X,Y,'dynamic')

In [57]:
dynamicxtrain.shape,dynamicytrain.shape

((2862, 25, 40), (2862, 4))

In [ ]:
####################################################################################################################################

In [58]:
dynamicxtrain.shape,dynamicxtest.shape,dynamicytrain.shape,dynamicytest.shape,dynamiclabels.shape

((2862, 25, 40), (1227, 25, 40), (2862, 4), (1227, 4), (1227,))

In [59]:
#xtrain,ytrain = resampleData(originalx,originaly)
staticxtrain,staticxtest,staticytrain,staticytest,staticlabels = splitData(X,Y,'static')
staticxtrain.shape,staticxtest.shape,staticytrain.shape,staticytest.shape,staticlabels.shape

((2991, 25, 40), (1282, 25, 40), (2991, 3), (1282, 3), (1282,))

**start** **transformers**

In [60]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

from matplotlib import pyplot
import matplotlib.pyplot as plt
import keras
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [29]:
%cd sample_data

/content/sample_data


In [61]:
dynamicxtrain.shape

(2862, 25, 40)

In [62]:
import model

learningRate = 3e-4

optimizer = tf.keras.optimizers.Adam(learningRate)
optimizer2 = tf.keras.optimizers.Adam(learningRate)

dynamic_classifier = model.HART((25,40),4)
static_classifier = model.HART((25,40),3)

dynamic_classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
static_classifier.compile(
    optimizer=optimizer2,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)

In [63]:
from sklearn.utils import class_weight

temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(dynamiclabels),
                                                 y = dynamiclabels.ravel())
class_weights = {j : temp_weights[j] for j in range(len(temp_weights))}

temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(staticlabels),
                                                 y = staticlabels.ravel())
class_weights2 = {j : temp_weights[j] for j in range(len(temp_weights))}

In [64]:
np.unique(staticlabels)


array([1, 2, 3], dtype=int32)

In [65]:
checkpoint_filepath = "bestValcheckpoint.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
)

history0 = dynamic_classifier.fit(
    x=dynamicxtrain,
    y=dynamicytrain,
  # validation_data = (dynamicxtest,dynamicytest),
    batch_size=256,
    epochs=100,
    verbose=1,
    class_weight=class_weights,
    callbacks=[checkpoint_callback],
)

history1 = static_classifier.fit(
    x=staticxtrain,
    y=staticytrain,
  # validation_data = (dynamicxtest,dynamicytest),
    batch_size=256,
    epochs=100,
    verbose=1,
    class_weight=class_weights2,
    callbacks=[checkpoint_callback],
)

Epoch 1/100
12/12 [==============================] - ETA: 0s - loss: 1.3204 - accuracy: 0.4119

12/12 [==============================] - 42s 228ms/step - loss: 1.3204 - accuracy: 0.4119
Epoch 2/100
12/12 [==============================] - ETA: 0s - loss: 1.0751 - accuracy: 0.6202

12/12 [==============================] - 2s 117ms/step - loss: 1.0751 - accuracy: 0.6202
Epoch 3/100
12/12 [==============================] - ETA: 0s - loss: 0.9010 - accuracy: 0.7369

12/12 [==============================] - 2s 138ms/step - loss: 0.9010 - accuracy: 0.7369
Epoch 4/100
12/12 [==============================] - ETA: 0s - loss: 0.7908 - accuracy: 0.7935

12/12 [==============================] - 2s 186ms/step - loss: 0.7908 - accuracy: 0.7935
Epoch 5/100
12/12 [==============================] - ETA: 0s - loss: 0.7284 - accuracy: 0.8312

...

12/12 [==============================] - 1s 55ms/step - loss: 0.2953 - accuracy: 0.9993
Epoch 97/100
11/12 [==========================>...] - ETA: 0s - loss: 0.2953 - accuracy: 0.9993

12/12 [==============================] - 1s 56ms/step - loss: 0.2952 - accuracy: 0.9993
Epoch 98/100
12/12 [==============================] - ETA: 0s - loss: 0.2995 - accuracy: 0.9977

12/12 [==============================] - 1s 55ms/step - loss: 0.2995 - accuracy: 0.9977
Epoch 99/100
12/12 [==============================] - ETA: 0s - loss: 0.3001 - accuracy: 0.9983

12/12 [==============================] - 1s 55ms/step - loss: 0.3001 - accuracy: 0.9983
Epoch 100/100
12/12 [==============================] - ETA: 0s - loss: 0.2965 - accuracy: 0.9997

12/12 [==============================] - 1s 55ms/step - loss: 0.2965 - accuracy: 0.9997


In [67]:
transitionxtrain,transitionxtest,transitionytrain,transitionytest,transitionlabels = splitData(X,Y,'transition')
transitionxtrain.shape,transitionxtest.shape,transitionytrain.shape,transitionytest.shape,transitionlabels.shape

((4232, 25, 40), (1815, 25, 40), (4232, 5), (1815, 5), (1815,))

In [68]:
temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(transitionlabels),
                                                 y = transitionlabels.ravel())
class_weights3 = {j : temp_weights[j] for j in range(len(temp_weights))}

optimizer3 = tf.keras.optimizers.Adam(learningRate)
transition_classifier = model.HART((25,40),5)
transition_classifier.compile(
    optimizer=optimizer3,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)

In [69]:
history3 = transition_classifier.fit(
    x=transitionxtrain,
    y=transitionytrain,
  # validation_data = (dynamicxtest,dynamicytest),
    batch_size=256,
    epochs=100,
    verbose=1,
    class_weight=class_weights3,
    callbacks=[checkpoint_callback],
)

Epoch 1/100
17/17 [==============================] - ETA: 0s - loss: 1.2173 - accuracy: 0.5673

17/17 [==============================] - 33s 70ms/step - loss: 1.2173 - accuracy: 0.5673
Epoch 2/100
17/17 [==============================] - ETA: 0s - loss: 0.8459 - accuracy: 0.7698

17/17 [==============================] - 1s 76ms/step - loss: 0.8459 - accuracy: 0.7698
Epoch 3/100
17/17 [==============================] - ETA: 0s - loss: 0.7173 - accuracy: 0.8526

17/17 [==============================] - 1s 78ms/step - loss: 0.7173 - accuracy: 0.8526
Epoch 4/100
17/17 [==============================] - ETA: 0s - loss: 0.6349 - accuracy: 0.8998

17/17 [==============================] - 1s 79ms/step - loss: 0.6349 - accuracy: 0.8998
Epoch 5/100
17/17 [==============================] - ETA: 0s - loss: 0.5744 - accuracy: 0.9284

...

17/17 [==============================] - 1s 54ms/step - loss: 0.3955 - accuracy: 0.9986
Epoch 97/100
17/17 [==============================] - ETA: 0s - loss: 0.3969 - accuracy: 0.9981

17/17 [==============================] - 1s 56ms/step - loss: 0.3969 - accuracy: 0.9981
Epoch 98/100
17/17 [==============================] - ETA: 0s - loss: 0.3996 - accuracy: 0.9972

17/17 [==============================] - 2s 121ms/step - loss: 0.3996 - accuracy: 0.9972
Epoch 99/100
17/17 [==============================] - ETA: 0s - loss: 0.4023 - accuracy: 0.9965

17/17 [==============================] - 3s 187ms/step - loss: 0.4023 - accuracy: 0.9965
Epoch 100/100
17/17 [==============================] - ETA: 0s - loss: 0.4001 - accuracy: 0.9981

17/17 [==============================] - 3s 143ms/step - loss: 0.4001 - accuracy: 0.9981


In [49]:
transscore = transition_classifier.evaluate(xtest, ytest)
print("traditional transformer model Accuracy: ", transscore[1])

136/136 [==============================] - 5s 23ms/step - loss: 0.6373 - accuracy: 0.9586
traditional transformer model Accuracy:  0.9585935473442078


In [70]:
yexpect = np.concatenate([dynamiclabels,staticlabels, transitionlabels], axis=0)

yexpect.shape

(4324,)

In [71]:
merge_arr = np.concatenate([dynamicxtest, staticxtest, transitionxtest], axis=0)


In [72]:
merge_arr.shape

(4324, 25, 40)

In [73]:
np.unique(dynamiclabels),np.unique(staticlabels),np.unique(transitionlabels)

(array([4, 5, 8, 9], dtype=int32),
 array([1, 2, 3], dtype=int32),
 array([ 0,  6,  7, 10, 11], dtype=int32))

In [74]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([4,5,8,9])
statics = np.array([1,2,3])
transitions = np.array([0,6,7,10,11])
for i in range(len(merge_arr)):#len(xtest)
    row = merge_arr[i].reshape(1,25,40)
    if(yexpect[i] in [4,5,8,9]):
        predict_x=dynamic_classifier.predict(row)

        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    elif(yexpect[i] in [1,2,3]):

        predict_x=static_classifier.predict(row)

        pred=statics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    else:

        predict_x=transition_classifier.predict(row)

        pred=transitions[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])


    print(i)
print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))

Streaming output truncated to the last 5000 lines.
1824
1/1 [==============================] - 0s 30ms/step
1825
1/1 [==============================] - 0s 31ms/step
1826
1/1 [==============================] - 0s 28ms/step
1827
1/1 [==============================] - 0s 29ms/step
1828
1/1 [==============================] - 0s 32ms/step
1829
1/1 [==============================] - 0s 28ms/step
1830
1/1 [==============================] - 0s 30ms/step
1831
1/1 [==============================] - 0s 31ms/step
1832
1/1 [==============================] - 0s 30ms/step
1833
1/1 [==============================] - 0s 32ms/step
1834
1/1 [==============================] - 0s 29ms/step
1835
1/1 [==============================] - 0s 28ms/step
1836
1/1 [==============================] - 0s 28ms/step
1837
1/1 [==============================] - 0s 30ms/step
1838
1/1 [==============================] - 0s 29ms/step
1839
1/1 [==============================] - 0s 39ms/step
1840
1/1 [===========================

In [76]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(yexpect, scores)*100))
# (static rf)(dynamic transformer)(transition rf) :  98.120%
# (static rf)(dynamic transformer)(transition transformer):  98.611 %

Accuracy score TEST: 97.849


**end** **transformers**

In [50]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(xtrain[0])
input_dim = len(xtrain[0][0])
n_classes =12

print(timesteps)
print(input_dim)
print(len(dynamicxtrain))



25
40
7095


In [51]:
epochs = 100
batch_size = 16
n_hidden = 64
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model1.add(Dropout(0.5))
model1.add(LSTM(32))
model1.add(Dropout(0.5))
model1.add(Dense(n_classes, activation='softmax'))
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 25, 64)            26880     
                                                                 
 dropout_35 (Dropout)        (None, 25, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_36 (Dropout)        (None, 32)                0         
                                                                 
 dense_70 (Dense)            (None, 12)                396       
                                                                 
Total params: 39692 (155.05 KB)
Trainable params: 39692 (155.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [53]:
model1.fit(xtrain,
          ytrain,
          batch_size=batch_size,
          #validation_data=(dynamicxtest, dynamicytest),
          epochs=epochs,
          )

Epoch 1/100
631/631 [==============================] - 9s 7ms/step - loss: 1.2612 - accuracy: 0.6242
Epoch 2/100
631/631 [==============================] - 6s 9ms/step - loss: 0.7042 - accuracy: 0.8021
Epoch 3/100
631/631 [==============================] - 5s 7ms/step - loss: 0.5286 - accuracy: 0.8564
Epoch 4/100
631/631 [==============================] - 4s 7ms/step - loss: 0.4456 - accuracy: 0.8781
Epoch 5/100
631/631 [==============================] - 5s 8ms/step - loss: 0.3699 - accuracy: 0.9027
Epoch 6/100
631/631 [==============================] - 5s 8ms/step - loss: 0.3272 - accuracy: 0.9165
Epoch 7/100
631/631 [==============================] - 5s 8ms/step - loss: 0.3082 - accuracy: 0.9250
Epoch 8/100
631/631 [==============================] - 5s 8ms/step - loss: 0.2817 - accuracy: 0.9301
Epoch 9/100
631/631 [==============================] - 6s 9ms/step - loss: 0.2526 - accuracy: 0.9386
Epoch 10/100
631/631 [==============================] - 4s 7ms/step - loss: 0.2364 - accura

In [54]:
score = model1.evaluate(xtest, ytest)
print("Accuracy: ", score[1])

136/136 [==============================] - 1s 4ms/step - loss: 0.1389 - accuracy: 0.9764
Accuracy:  0.9764052629470825


In [ ]:
predict_x=model1.predict(dynamicxtest)
ypred=np.argmax(predict_x,axis=1)

29/29 [==============================] - 1s 5ms/step


In [ ]:
ypred.shape, dynamiclabels.shape

((918,), (918,))

In [ ]:

ypred[ypred == 2] = 3
ypred[ypred == 1] = 2
ypred[ypred == 0] = 1
ypred

array([3, 2, 3, 3, 1, 2, 3, 3, 1, 2, 2, 3, 3, 3, 1, 1, 1, 3, 1, 3, 3, 2,
       1, 1, 1, 2, 2, 3, 2, 3, 3, 2, 3, 1, 2, 2, 1, 1, 3, 2, 1, 3, 3, 1,
       2, 1, 2, 3, 1, 1, 3, 2, 3, 2, 2, 3, 1, 3, 1, 3, 3, 1, 2, 2, 3, 3,
       2, 3, 2, 1, 2, 2, 3, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 3, 1, 3, 2, 1,
       2, 3, 3, 3, 3, 3, 2, 1, 3, 1, 3, 1, 2, 2, 1, 2, 2, 1, 1, 3, 2, 2,
       3, 1, 1, 2, 1, 2, 3, 3, 3, 3, 3, 1, 1, 3, 2, 2, 1, 1, 1, 2, 1, 2,
       3, 1, 1, 2, 3, 1, 2, 2, 1, 2, 3, 2, 3, 3, 2, 1, 3, 2, 2, 1, 2, 3,
       1, 1, 1, 1, 1, 2, 2, 3, 2, 2, 2, 1, 1, 1, 1, 2, 3, 1, 2, 3, 1, 3,
       2, 2, 3, 2, 1, 3, 3, 1, 3, 2, 2, 3, 2, 1, 2, 3, 2, 3, 2, 3, 1, 2,
       1, 3, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 3, 3, 1,
       2, 3, 3, 3, 1, 2, 1, 3, 3, 3, 3, 3, 1, 2, 2, 1, 1, 1, 1, 3, 3, 3,
       1, 2, 2, 2, 3, 3, 1, 2, 1, 3, 2, 3, 2, 2, 2, 3, 3, 2, 2, 1, 1, 1,
       2, 2, 3, 2, 3, 3, 1, 1, 2, 2, 1, 3, 1, 3, 2, 3, 2, 2, 3, 2, 3, 3,
       3, 3, 1, 3, 3, 3, 3, 1, 3, 2, 1, 2, 2, 2, 1,

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, dynamiclabels)*100))

Accuracy score TEST: 99.020


**static**

In [ ]:
xtrain,ytrain = resampleData(originalx,originaly)
staticxtrain,staticxtest,staticytrain,staticytest,staticlabels = splitData(xtrain,ytrain,'static')
staticxtrain.shape,staticxtest.shape,staticytrain.shape,staticytest.shape,staticlabels.shape

((2139, 128, 2, 3), (918, 128, 2, 3), (2139, 3), (918, 3), (918,))

In [ ]:
staticxtrain =staticxtrain.reshape(2139,128,6)
staticxtest = staticxtest.reshape(918,128,6)

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(staticxtrain[0])
input_dim = len(staticxtrain[0][0])
n_classes =3

print(timesteps)
print(input_dim)
print(len(staticxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model2 = Sequential()
# Configuring the parameters
model2.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model2.add(Dropout(0.5))
model2.add(LSTM(32))
model2.add(Dropout(0.5))
model2.add(Dense(n_classes, activation='softmax'))
model2.summary()

128
6
2139
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_2 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 30,691
Trainable params: 30,691
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model2.fit(staticxtrain,
          staticytrain,
          batch_size=batch_size,
          #validation_data=(staticxtest, staticytest),
          epochs=epochs,
          )

Epoch 1/100
134/134 [==============================] - 5s 10ms/step - loss: 0.4238 - accuracy: 0.8125
Epoch 2/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2870 - accuracy: 0.8822
Epoch 3/100
134/134 [==============================] - 2s 13ms/step - loss: 0.3133 - accuracy: 0.8822
Epoch 4/100
134/134 [==============================] - 1s 10ms/step - loss: 0.3008 - accuracy: 0.8841
Epoch 5/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2722 - accuracy: 0.8920
Epoch 6/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2893 - accuracy: 0.8925
Epoch 7/100
134/134 [==============================] - 2s 13ms/step - loss: 0.2554 - accuracy: 0.8957
Epoch 8/100
134/134 [==============================] - 2s 14ms/step - loss: 0.2848 - accuracy: 0.8976
Epoch 9/100
134/134 [==============================] - 1s 11ms/step - loss: 0.2457 - accuracy: 0.9018
Epoch 10/100
134/134 [==============================] - 1s 10ms/step - loss: 0.253

In [ ]:
predict_x=model2.predict(staticxtest)
ypred=np.argmax(predict_x,axis=1)


ypred[ypred == 2] = 6
ypred[ypred == 1] = 5
ypred[ypred == 0] = 4
ypred

from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, staticlabels)*100))

29/29 [==============================] - 0s 7ms/step
Accuracy score TEST: 91.612


In [ ]:
score = model2.evaluate(staticxtest, staticytest)
print("Accuracy: ", score[1])

29/29 [==============================] - 1s 9ms/step - loss: 0.2233 - accuracy: 0.9161
Accuracy:  0.9161220192909241


**primary**

In [ ]:
primaryxtrain,primaryxtest,primaryytrain,primaryytest,primarylabels = splitData(xtrain,ytrain,'primary')
primaryxtrain.shape,primaryxtest.shape,primaryytrain.shape,primaryytest.shape,primarylabels.shape

((4279, 128, 2, 3), (1835, 128, 2, 3), (4279, 6), (1835, 6), (1835,))

In [ ]:
primaryxtrain =primaryxtrain.reshape(4279,128,6)
primaryxtest = primaryxtest.reshape(1835,128,6)

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(primaryxtrain[0])
input_dim = len(primaryxtrain[0][0])
n_classes =6

print(timesteps)
print(input_dim)
print(len(primaryxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model0 = Sequential()
# Configuring the parameters
model0.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model0.add(Dropout(0.5))
model0.add(LSTM(32))
model0.add(Dropout(0.5))
model0.add(Dense(n_classes, activation='softmax'))
model0.summary()

128
6
4279
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_2 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 30,790
Trainable params: 30,790
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model0.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model0.fit(primaryxtrain,
          primaryytrain,
          batch_size=batch_size,
          #validation_data=(transitionxtest, transitionytest),
          epochs=epochs,
          )

Epoch 1/100
268/268 [==============================] - 6s 10ms/step - loss: 1.2123 - accuracy: 0.4940
Epoch 2/100
268/268 [==============================] - 3s 10ms/step - loss: 0.8864 - accuracy: 0.6658
Epoch 3/100
268/268 [==============================] - 3s 12ms/step - loss: 0.6335 - accuracy: 0.7904
Epoch 4/100
268/268 [==============================] - 3s 11ms/step - loss: 0.4913 - accuracy: 0.8434
Epoch 5/100
268/268 [==============================] - 3s 10ms/step - loss: 0.4535 - accuracy: 0.8518
Epoch 6/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3716 - accuracy: 0.8827
Epoch 7/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3489 - accuracy: 0.8965
Epoch 8/100
268/268 [==============================] - 4s 14ms/step - loss: 0.4476 - accuracy: 0.8733
Epoch 9/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3018 - accuracy: 0.9149
Epoch 10/100
268/268 [==============================] - 3s 10ms/step - loss: 0.268

**transition**

In [ ]:
transitionxtrain,transitionxtest,transitionytrain,transitionytest,transitionlabels = splitData(xtrain,ytrain,'transition')
transitionxtrain.shape,transitionxtest.shape,transitionytrain.shape,transitionytest.shape,transitionlabels.shape

((4279, 128, 2, 3), (1835, 128, 2, 3), (4279, 6), (1835, 6), (1835,))

In [ ]:
transitionxtrain =transitionxtrain.reshape(4279,128,6)
transitionxtest = transitionxtest.reshape(1835,128,6)

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(transitionxtrain[0])
input_dim = len(transitionxtrain[0][0])
n_classes =6

print(timesteps)
print(input_dim)
print(len(transitionxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model3 = Sequential()
# Configuring the parameters
model3.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model3.add(Dropout(0.5))
model3.add(LSTM(32))
model3.add(Dropout(0.5))
model3.add(Dense(n_classes, activation='softmax'))
model3.summary()

128
6
4279
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_4 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 30,790
Trainable params: 30,790
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model3.fit(transitionxtrain,
          transitionytrain,
          batch_size=batch_size,
          #validation_data=(transitionxtest, transitionytest),
          epochs=epochs,
          )

Epoch 1/100
268/268 [==============================] - 7s 13ms/step - loss: 1.0277 - accuracy: 0.5691
Epoch 2/100
268/268 [==============================] - 3s 10ms/step - loss: 0.7277 - accuracy: 0.7018
Epoch 3/100
268/268 [==============================] - 3s 10ms/step - loss: 0.6189 - accuracy: 0.7689
Epoch 4/100
268/268 [==============================] - 3s 10ms/step - loss: 0.5205 - accuracy: 0.8074
Epoch 5/100
268/268 [==============================] - 3s 12ms/step - loss: 0.4553 - accuracy: 0.8317
Epoch 6/100
268/268 [==============================] - 3s 13ms/step - loss: 0.4423 - accuracy: 0.8500
Epoch 7/100
268/268 [==============================] - 3s 11ms/step - loss: 0.3886 - accuracy: 0.8696
Epoch 8/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3626 - accuracy: 0.8726
Epoch 9/100
268/268 [==============================] - 3s 11ms/step - loss: 0.3433 - accuracy: 0.8843
Epoch 10/100
268/268 [==============================] - 4s 14ms/step - loss: 0.307

In [ ]:
predict_x=model3.predict(transitionxtest)
ypred=np.argmax(predict_x,axis=1)


ypred[ypred == 5] = 12
ypred[ypred == 4] = 11
ypred[ypred == 3] = 10
ypred[ypred == 2] = 9
ypred[ypred == 1] = 8
ypred[ypred == 0] = 7
ypred

58/58 [==============================] - 1s 7ms/step


array([12, 11, 10, ..., 12, 10, 10])

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, transitionlabels)*100))

Accuracy score TEST: 93.842


**master lstm start**

In [ ]:
xtrain2,xtest2,ytrain2,ytest2,labels2 = splitData(xtrain,ytrain,'all')


In [ ]:
xtrain2.shape,xtest2.shape

((8559, 128, 2, 3), (3669, 128, 2, 3))

In [ ]:
xtrain2 = xtrain2.reshape(8559,128,6)
xtest2 = xtest2.reshape(3669,128,6)

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(xtrain2[0])
input_dim = len(xtrain2[0][0])
n_classes =12

print(timesteps)
print(input_dim)
print(len(transitionxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model0 = Sequential()
# Configuring the parameters
model0.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model0.add(Dropout(0.5))
model0.add(LSTM(32))
model0.add(Dropout(0.5))
model0.add(Dense(n_classes, activation='softmax'))
model0.summary()

128
6
4279
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_8 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_9 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_9 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 12)                396       
                                                                 
Total params: 30,988
Trainable params: 30,988
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model0.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model0.fit(xtrain2,
          ytrain2,
          batch_size=batch_size,
          #validation_data=(transitionxtest, transitionytest),
          epochs=epochs,
          )

Epoch 1/100
535/535 [==============================] - 10s 12ms/step - loss: 1.6680 - accuracy: 0.3667
Epoch 2/100
535/535 [==============================] - 5s 10ms/step - loss: 1.1381 - accuracy: 0.5903
Epoch 3/100
535/535 [==============================] - 6s 12ms/step - loss: 0.9207 - accuracy: 0.6787
Epoch 4/100
535/535 [==============================] - 5s 10ms/step - loss: 0.7698 - accuracy: 0.7466
Epoch 5/100
535/535 [==============================] - 6s 11ms/step - loss: 0.6722 - accuracy: 0.7914
Epoch 6/100
535/535 [==============================] - 6s 12ms/step - loss: 0.6205 - accuracy: 0.8163
Epoch 7/100
535/535 [==============================] - 6s 10ms/step - loss: 0.5399 - accuracy: 0.8335
Epoch 8/100
535/535 [==============================] - 6s 12ms/step - loss: 0.5122 - accuracy: 0.8464
Epoch 9/100
535/535 [==============================] - 6s 10ms/step - loss: 0.4623 - accuracy: 0.8641
Epoch 10/100
535/535 [==============================] - 6s 12ms/step - loss: 0.42

In [ ]:
score = model0.evaluate(xtest2, ytest2)
print("Accuracy: ", score[1])

115/115 [==============================] - 2s 8ms/step - loss: 0.1004 - accuracy: 0.9717
Accuracy:  0.9716544151306152


**master lstm end**

In [ ]:
xtrain,ytrain = resampleData(originalx,originaly)

xtrain,xtest,ytrain,ytest,ylabels = splitData(xtrain,ytrain,'all')
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

((8559, 128, 2, 3), (8559, 12), (3669, 128, 2, 3), (3669, 12))

In [ ]:
predict_x=model1.predict(dynamicxtest)
ypred1=np.argmax(predict_x,axis=1)


ypred1[ypred1 == 2] = 3
ypred1[ypred1 == 1] = 2
ypred1[ypred1 == 0] = 1


predict_x=model2.predict(staticxtest)
ypred2=np.argmax(predict_x,axis=1)


ypred2[ypred2 == 2] = 6
ypred2[ypred2 == 1] = 5
ypred2[ypred2 == 0] = 4


predict_x=model3.predict(transitionxtest)
ypred3=np.argmax(predict_x,axis=1)


ypred3[ypred3 == 5] = 12
ypred3[ypred3 == 4] = 11
ypred3[ypred3 == 3] = 10
ypred3[ypred3 == 2] = 9
ypred3[ypred3 == 1] = 8
ypred3[ypred3 == 0] = 7


58/58 [==============================] - 0s 5ms/step


In [ ]:
ypred1.shape,ypred2.shape,ypred3.shape

((918,), (918,), (1835,))

In [ ]:
ypred = np.concatenate([ypred1, ypred2, ypred3], axis=0)
ypred.shape

(3671,)

In [ ]:
ypred

array([ 3,  2,  3, ..., 12, 10, 10])

In [ ]:
yexpect = np.concatenate([dynamiclabels,staticlabels, transitionlabels], axis=0)

yexpect.shape

(3671,)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, yexpect)*100))

Accuracy score TEST: 96.949


In [ ]:
merge_arr = np.concatenate([dynamicxtest, staticxtest, transitionxtest], axis=0)


In [ ]:
merge_arr.shape

(3671, 128, 6)

In [ ]:
predict_x=model0.predict(merge_arr)
ypred3=np.argmax(predict_x,axis=1)


ypred3[ypred3 == 11] = 12
ypred3[ypred3 == 10] = 11
ypred3[ypred3 == 9] = 10
ypred3[ypred3 == 8] = 9
ypred3[ypred3 == 7] = 8
ypred3[ypred3 == 6] = 7
ypred3[ypred3 == 5] = 6
ypred3[ypred3 == 4] = 5
ypred3[ypred3 == 3] = 4
ypred3[ypred3 == 2] = 3
ypred3[ypred3 == 1] = 2
ypred3[ypred3 == 0] = 1

115/115 [==============================] - 1s 5ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred3, yexpect)*100))

Accuracy score TEST: 46.391


In [ ]:
ypred3.shape,merge_arr.shape

((3671,), (3671, 128, 6))

In [ ]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
transitions = np.array([7,8,9,10,11,12])
for i in range(len(merge_arr)):#len(xtest)
    row = merge_arr[i].reshape(1,128,6)
    if(rf_pred[i] in [1,2,3]):
        predict_x=model1.predict(row)

        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    elif(rf_pred[i] in [4,5,6]):
        predict_x=model2.predict(row)

        pred=statics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    else:
        predict_x=model3.predict(row)

        pred=transitions[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])


    print(i)
print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))

Streaming output truncated to the last 5000 lines.
1171
1/1 [==============================] - 0s 20ms/step
1172
1/1 [==============================] - 0s 20ms/step
1173
1/1 [==============================] - 0s 22ms/step
1174
1/1 [==============================] - 0s 19ms/step
1175
1/1 [==============================] - 0s 22ms/step
1176
1/1 [==============================] - 0s 21ms/step
1177
1/1 [==============================] - 0s 20ms/step
1178
1/1 [==============================] - 0s 20ms/step
1179
1/1 [==============================] - 0s 21ms/step
1180
1/1 [==============================] - 0s 20ms/step
1181
1/1 [==============================] - 0s 22ms/step
1182
1/1 [==============================] - 0s 21ms/step
1183
1/1 [==============================] - 0s 27ms/step
1184
1/1 [==============================] - 0s 24ms/step
1185
1/1 [==============================] - 0s 23ms/step
1186
1/1 [==============================] - 0s 20ms/step
1187
1/1 [===========================

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(yexpect, scores)*100))

Accuracy score TEST: 97.548


In [ ]:
yexpect,Y_test

(array([ 3,  2,  3, ..., 12, 10, 10], dtype=uint32),
 array([ 3,  2,  3, ..., 12, 10, 10], dtype=uint32))

In [ ]:
#score = model1.evaluate(dynamicxtest, dynamicytest)
#print("dynamic Accuracy: ", score[1])

#score = model2.evaluate(staticxtest, staticytest)
#print("static Accuracy: ", score[1])

score = model3.evaluate(transitionxtest, transitionytest)
print("transition Accuracy: ", score[1])

score = model0.evaluate(primaryxtest, primaryytest)
print("primary Accuracy: ", score[1])

58/58 [==============================] - 4s 9ms/step - loss: 0.0388 - accuracy: 0.9929
transition Accuracy:  0.9929155111312866
58/58 [==============================] - 1s 7ms/step - loss: 0.1104 - accuracy: 0.9515
primary Accuracy:  0.9514986276626587


In [ ]:
transitionxtest2 = xtest[(ylabels==7)|(ylabels==8)|(ylabels==9)|(ylabels==10)|(ylabels==11)|(ylabels==12)]
transitionytest2 = ytest[(ylabels==7)|(ylabels==8)|(ylabels==9)|(ylabels==10)|(ylabels==11)|(ylabels==12)]
transitionlabels2 = ylabels[(ylabels==7)|(ylabels==8)|(ylabels==9)|(ylabels==10)|(ylabels==11)|(ylabels==12)]

transitionxtest2.shape,transitionytest2.shape

((1833, 128, 6), (1833, 12))

In [ ]:
transitionxtest2 = transitionxtest2.reshape(1833,128,6)

In [ ]:
transitionxtest2.shape

(1833, 128, 6)

In [ ]:
score = model3.evaluate(transitionxtest2[:100], transitionytest2[:100])
print("transition Accuracy: ", score[1])

4/4 [==============================] - 0s 9ms/step - loss: 12.2473 - accuracy: 0.1300
transition Accuracy:  0.12999999523162842


In [ ]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
primary = np.array([1,2,3,4,5,6])
len =1833 #len(xtest)

transitions = np.array([7,8,9,10,11,12])
for i in range(len):#len(xtest)
    row = transitionxtest2[i].reshape(1,128,6)
    predict_x=model3.predict(row)
    pred=transitions[np.argmax(predict_x,axis=1)]
    scores.append(pred[0])


    print(i)


1/1 [==============================] - 0s 24ms/step
0
1/1 [==============================] - 0s 21ms/step
1
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 21ms/step
3
1/1 [==============================] - 0s 23ms/step
4
1/1 [==============================] - 0s 24ms/step
5
1/1 [==============================] - 0s 23ms/step
6
1/1 [==============================] - 0s 20ms/step
7
1/1 [==============================] - 0s 28ms/step
8
1/1 [==============================] - 0s 21ms/step
9
1/1 [==============================] - 0s 22ms/step
10
1/1 [==============================] - 0s 24ms/step
11
1/1 [==============================] - 0s 24ms/step
12
1/1 [==============================] - 0s 20ms/step
13
1/1 [==============================] - 0s 20ms/step
14
1/1 [==============================] - 0s 28ms/step
15
1/1 [==============================] - 0s 22ms/step
16
1/1 [==============================] - 0s 23ms/step
17
1/1 [===============

In [ ]:
print('Accuracy score TEST: {:.3f}'.format(accuracy_score(transitionlabels2, scores)*100))

Accuracy score TEST: 36.388


**Final**

In [ ]:
xtrain,ytrain = resampleData(originalx,originaly)

xtrain,xtest,ytrain,ytest,ylabels = splitData(xtrain,ytrain,'all')
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

((8559, 128, 2, 3), (8559, 12), (3669, 128, 2, 3), (3669, 12))

In [ ]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
transitions = np.array([7,8,9,10,11,12])
for i in range(len(xtest)):#len(xtest)
    row = xtest[i].reshape(1,128,6)
    if(ylabels[i] in [1,2,3]):
        predict_x=model1.predict(row)

        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])
        print('dynamic classifier : pred: '+str(pred[0])+' actual: '+str(ylabels[i]))
        if(pred[0]!= ylabels[i]):
          dynamiccount = dynamiccount+1
    elif(ylabels[i] in [4,5,6]):
        predict_x=model2.predict(row)

        pred=statics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])
        print('static classifier : pred: '+str(pred[0])+' actual: '+str(ylabels[i]))
        if(pred[0]!= ylabels[i]):
          staticcount = staticcount+1
    else:
        predict_x=model3.predict(row)

        pred=transitions[np.argmax(predict_x,axis=1)]
       # pred[0] = ylabels[i]
        scores.append(pred[0])
        print('transition classifier : pred: '+str(pred[0])+' actual: '+str(ylabels[i]))
        if(pred[0]!= ylabels[i]):
          transitioncount = transitioncount+1

    print(i)
print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))

Streaming output truncated to the last 5000 lines.
2002
1/1 [==============================] - 0s 28ms/step
static classifier : pred: 6 actual: 6
2003
1/1 [==============================] - 0s 26ms/step
static classifier : pred: 6 actual: 6
2004
1/1 [==============================] - 0s 27ms/step
transition classifier : pred: 10 actual: 7
2005
1/1 [==============================] - 0s 24ms/step
dynamic classifier : pred: 3 actual: 3
2006
1/1 [==============================] - 0s 24ms/step
transition classifier : pred: 12 actual: 10
2007
1/1 [==============================] - 0s 27ms/step
static classifier : pred: 6 actual: 6
2008
1/1 [==============================] - 0s 24ms/step
transition classifier : pred: 10 actual: 10
2009
1/1 [==============================] - 0s 25ms/step
transition classifier : pred: 10 actual: 8
2010
1/1 [==============================] - 0s 25ms/step
dynamic classifier : pred: 1 actual: 2
2011
1/1 [==============================] - 0s 24ms/step
transition cl

In [ ]:
scores=np.array(scores)
scores.shape

(3669,)

In [ ]:
ylabels.shape

(3669,)

In [ ]:

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ylabels, scores)*100))

Accuracy score TEST: 55.492


In [ ]:
np.unique(ylabels, return_counts=True)


(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=uint32),
 array([280, 294, 299, 315, 322, 326, 310, 322, 295, 307, 310, 289]))